In [2]:
import pandas as pd
import re

In [3]:
liste_article_octobre = pd.read_excel(r'C:\Users\Hanaa\Desktop\PETITFUTE\data\exports\listes_article_octobre.xlsx')

In [4]:
base_URL = liste_article_octobre[['URL']]

In [ ]:
# ### Code pour scraper les URL VO
# import nest_asyncio
# nest_asyncio.apply()

# import asyncio
# import pandas as pd
# from urllib.parse import urlparse, parse_qs, unquote
# from bs4 import BeautifulSoup
# from playwright.async_api import async_playwright


# # ==========================================================
# # 1️⃣ Fonction Playwright pour charger une page dynamiquement
# # ==========================================================
# async def get_dynamic_html(url):
#     """Rend la page (avec JS) et renvoie le HTML final."""
#     async with async_playwright() as p:
#         browser = await p.chromium.launch(headless=True)
#         context = await browser.new_context(
#             user_agent=(
#                 "Mozilla/5.0 (Windows NT 10.0; Win64; x64) "
#                 "AppleWebKit/537.36 (KHTML, like Gecko) "
#                 "Chrome/127.0.0.0 Safari/537.36"
#             )
#         )

#         page = await context.new_page()

#         # 🔇 désactiver images et pubs pour rapidité
#         await page.route("**/*", lambda route: (
#             route.continue_()
#             if route.request.resource_type in ["document", "script", "xhr", "fetch"]
#             else route.abort()
#         ))

#         try:
#             await page.goto(url, timeout=90000, wait_until="domcontentloaded")
#             await page.wait_for_timeout(4000)
#             html = await page.content()
#         except Exception as e:
#             print(f"⚠️ Erreur sur {url}: {e}")
#             html = ""
#         await browser.close()
#         return html


# # ==========================================================
# # 2️⃣ Fonction d’extraction des liens GetYourGuide & Booking
# # ==========================================================
# def extract_partner_links(html):
#     """Extrait tous les liens GetYourGuide et Booking d’un HTML."""
#     soup = BeautifulSoup(html, "html.parser")

#     links_getyourguide = []
#     links_booking = []

#     for a in soup.find_all("a", href=True):
#         href = a["href"].strip().lower()
#         if "getyourguide" in href or "booking.com" in href:
#             # décoder si lien intermédiaire (urldeeplink)
#             parsed = urlparse(a["href"])
#             qs = parse_qs(parsed.query)
#             if "urldeeplink" in qs:
#                 decoded = unquote(qs["urldeeplink"][0])
#             else:
#                 decoded = a["href"]

#             if "getyourguide" in decoded.lower():
#                 links_getyourguide.append(decoded)
#             elif "booking.com" in decoded.lower():
#                 links_booking.append(decoded)

#     return links_getyourguide, links_booking


# # ==========================================================
# # 3️⃣ Fonction principale de traitement sur un DataFrame
# # ==========================================================
# async def enrich_dataframe_with_links(df, url_col="URL"):
#     """Ajoute deux colonnes au DataFrame : getyourguide_links et booking_links"""
#     results_g = []
#     results_b = []

#     for i, url in enumerate(df[url_col], 1):
#         print(f"\n[{i}/{len(df)}] Traitement de {url} ...")
#         html = await get_dynamic_html(url)
#         gyg, booking = extract_partner_links(html)
#         results_g.append(gyg)
#         results_b.append(booking)

#     df["getyourguide_links"] = results_g
#     df["booking_links"] = results_b
#     return df


# # ==========================================================
# # 4️⃣ Exemple d’utilisation
# # ==========================================================
# # 🧮 Exemple de table de départ

# # ⚡ Lancer le traitement
# #df_final = asyncio.get_event_loop().run_until_complete(enrich_dataframe_with_links(base_URL))
# #

In [8]:
!python scraping_petitfute.py


🚀 Lancement du scraping des liens partenaires Petit Futé...


[1/99] Traitement de https://www.petitfute.com/v62250-kew/actualite/m17-top-10-insolites-voyage/a11635-que-faire-a-londres-a-noel-les-14-activites-incontournables.html ...

[2/99] Traitement de https://www.petitfute.com/v48299-bangli/actualite/m17-top-10-insolites-voyage/a11947-les-10-etapes-incontournables-pour-une-escapade-a-bali.html ...

[3/99] Traitement de https://www.petitfute.com/v43490-caldera-de-taburiente/actualite/m17-top-10-insolites-voyage/a12467-que-faire-aux-canaries-top-15-des-experiences-a-ne-pas-manquer.html ...

[4/99] Traitement de https://www.petitfute.com/v49846-genes/actualite/m17-top-10-insolites-voyage/a13620-que-faire-a-genes-les-17-incontournables-a-visiter.html ...

[5/99] Traitement de https://www.petitfute.com/v48986-copenhague/actualite/m17-top-10-insolites-voyage/a18238-les-incontournables-de-copenhague.html ...

[6/99] Traitement de https://www.petitfute.com/v48159-marrakech/actualite/m17-to

In [86]:
df_final = pd.read_excel(r"C:\Users\Hanaa\Desktop\PETITFUTE\data\outputs\liens_partenaire_octobre.xlsx")
df_final["getyourguide_links"] = df_final["getyourguide_links"].apply(
    lambda x: ast.literal_eval(x) if isinstance(x, str) else x
)

In [87]:


def extract_gyg_ids_from_list(links):
    """Extrait tous les identifiants GetYourGuide (t + 5 ou 6 chiffres) d'une liste d'URLs."""
    if not isinstance(links, list):
        return []
    ids = []
    for url in links:
        # Cherche un motif 't' suivi de 5 ou 6 chiffres
        match = re.search(r"t\d{5,6}", url)
        if match:
            ids.append(match.group(0))
    return ids

In [88]:
# ✅ Appliquer la fonction à ta colonne existante
df_final["id_getyourguide_link"] = df_final["getyourguide_links"].apply(extract_gyg_ids_from_list)

In [89]:
df_final

,URL,getyourguide_links,booking_links,id_getyourguide_link
0,https://www.petitfute.com/v62250-kew/actualite...,[https://www.getyourguide.fr/londres-l57/londr...,['https://www.booking.com/hotel/gb/ibis-london...,"[t41419, t129039, t31392, t310718, t34660, t38..."
1,https://www.petitfute.com/v48299-bangli/actual...,[https://www.getyourguide.fr/seminyak-l95096/k...,['https://www.booking.com/hotel/id/sanak-retre...,"[t791280, t162114, t393678, t187641, t55668, t..."
2,https://www.petitfute.com/v43490-caldera-de-ta...,[https://www.getyourguide.fr/tenerife-l350/vis...,[],"[t51524, t417569, t206580, t58006, t534639, t1..."
3,https://www.petitfute.com/v49846-genes/actuali...,[https://www.getyourguide.fr/genes-l1009/visit...,['https://www.booking.com/hotel/it/ostellin-ge...,"[t622176, t13191, t50734, t29705, t399086, t81..."
4,https://www.petitfute.com/v48986-copenhague/ac...,[https://www.getyourguide.fr/faelledparken-l12...,['https://www.booking.com/hotel/dk/steel-house...,"[t72048, t66660, t504350, t168407, t37848, t22..."
...,...,...,...,...
94,https://www.petitfute.com/v42470-bodrum/actual...,[https://www.getyourguide.fr/bodrum-l846/tour-...,['https://www.booking.com/hotel/tr/seckin-kona...,"[t653504, t103136, t476435, t467340]"
95,https://www.petitfute.com/v42420-antalya/actua...,[https://www.getyourguide.fr/antalya-l172/anta...,['https://www.booking.com/hotel/tr/argos.fr.ht...,"[t950470, t469125, t468561, t975032, t949847, ..."
96,https://www.petitfute.com/v41475-chiang-rai/ac...,[https://www.getyourguide.fr/bangkok-l169/bang...,['https://www.booking.com/hotel/th/baan-noppaw...,"[t417016, t425854, t128406, t417017, t52629, t..."
97,https://www.petitfute.com/v48393-seoul/actuali...,[https://www.getyourguide.fr/seoul-l197/palais...,['https://www.booking.com/hotel/kr/grand-hyatt...,"[t467593, t63107, t113551, t492074, t440500, t..."


In [147]:
#data_gyg = pd.read_csv(r'C:\Users\Hanaa\Desktop\PETITFUTE\data\exports\Booking Overview.csv')
data_gyg = pd.read_excel(r'C:\Users\Hanaa\Desktop\PETITFUTE\data\exports\get-your-guide-gxcSWuH9.xlsx')

c:\Users\Hanaa\AppData\Local\Programs\Python\Python312\Lib\site-packages\openpyxl\styles\stylesheet.py:237: UserWarning: Workbook contains no default style, apply openpyxl's default
  warn("Workbook contains no default style, apply openpyxl's default")


In [148]:
data_gyg_activity = data_gyg[['View activity', 'Potential income']]

In [149]:
def extract_gyg_id(url):
    """Retourne l’identifiant GetYourGuide (t + 5 ou 6 chiffres) trouvé dans l’URL."""
    if not isinstance(url, str):
        return None
    match = re.search(r"t\d{5,6}", url)
    return match.group(0) if match else None

# Appliquer la fonction
data_gyg_activity["id_gyg"] = data_gyg_activity["View activity"].apply(extract_gyg_id)

C:\Users\Hanaa\AppData\Local\Temp\ipykernel_14608\1081691221.py:9: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data_gyg_activity["id_gyg"] = data_gyg_activity["View activity"].apply(extract_gyg_id)


In [150]:
data_gyg_activity["Potential income"] = pd.to_numeric(
    data_gyg_activity["Potential income"], errors="coerce"
).fillna(0)


C:\Users\Hanaa\AppData\Local\Temp\ipykernel_14608\530048308.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data_gyg_activity["Potential income"] = pd.to_numeric(


In [151]:
# --- Fonction pour calculer la somme du revenu GYG par ligne ---
def calc_revenu_gyg(ids, df_ref):
    """Somme des potential_income pour une liste d'IDs à partir du df_ref."""
    if not isinstance(ids, list) or len(ids) == 0:
        return 0.0
    subset = df_ref[df_ref["id_gyg"].isin(ids)]
    return subset["Potential income"].sum()

# --- Application à la table Petit Futé ---
df_final["revenu_gyg"] = df_final["id_getyourguide_link"].apply(lambda ids: calc_revenu_gyg(ids, data_gyg_activity))



In [152]:
print(df_final['revenu_gyg'].sum())
df_final['revenu_gyg'].value_counts()


172.92000000000002


revenu_gyg
0.00     86
6.72      2
6.45      1
7.51      1
4.06      1
71.68     1
2.80      1
4.48      1
10.50     1
8.40      1
31.36     1
11.40     1
0.84      1
Name: count, dtype: int64

C:\Users\Hanaa\AppData\Local\Temp\ipykernel_14608\1193131284.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data_gyg_activity["Potential income"] = pd.to_numeric(


dtype('int64')

In [101]:
df_final['booking_links'][0]

"['https://www.booking.com/hotel/gb/ibis-london-blackfriars.fr.html?aid=356980&label=gog235jc-1FCAsoUEIXaWJpcy1sb25kb24tYmxhY2tmcmlhcnNIDVgDaE2IAQGYAQ24AQfIAQzYAQHoAQH4AQKIAgGoAgO4AqKo8LcGwAIB0gIkMjA5N2ZjNjgtZmRhYi00MzRlLThmOWYtOTYwMmFhOGRiNDk12AIF4AIB&sid=a0ed4f8119aafb08ddb2c34d69e8b88d&age=0;all_sr_blocks=42636801_94463129_2_42_0%2C42636801_94463129_2_42_0;checkin=2024-10-06;checkout=2024-10-07;dest_id=-2601889;dest_type=city;dist=0;group_adults=2;group_children=1;hapos=1;highlighted_blocks=42636801_94463129_2_42_0%2C42636801_94463129_2_42_0;hpos=1;matching_block_id=42636801_94463129_2_42_0;no_rooms=1;req_adults=2;req_age=0;req_children=1;room1=A%2CA%2C0;sb_price_type=total;sr_order=popularity;sr_pri_blocks=42636801_94463129_2_42_0__12834%2C42636801_94463129_2_42_0__12834;srepoch=1727796262;srpvid=6b066c51ed76067f;type=total;ucfs=1&#map_opened-hotel_address', 'https://www.booking.com/hotel/gb/ibis-london-blackfriars.fr.html?aid=2324560&no_rooms=1&group_adults=2&label=actu-londres-no

# BOOKING

In [11]:
base_URL

,URL
0,https://www.petitfute.com/v62250-kew/actualite...
1,https://www.petitfute.com/v48299-bangli/actual...
2,https://www.petitfute.com/v43490-caldera-de-ta...
3,https://www.petitfute.com/v49846-genes/actuali...
4,https://www.petitfute.com/v48986-copenhague/ac...
...,...
94,https://www.petitfute.com/v42470-bodrum/actual...
95,https://www.petitfute.com/v42420-antalya/actua...
96,https://www.petitfute.com/v41475-chiang-rai/ac...
97,https://www.petitfute.com/v48393-seoul/actuali...


In [13]:
df_liens = pd.read_excel(r"C:\Users\Hanaa\Desktop\PETITFUTE\data\outputs\liens_partenaire_octobre.xlsx")
df_liens.head()

,URL,getyourguide_links,booking_links
0,https://www.petitfute.com/v62250-kew/actualite...,['https://www.getyourguide.fr/londres-l57/lond...,['https://www.booking.com/hotel/gb/ibis-london...
1,https://www.petitfute.com/v48299-bangli/actual...,['https://www.getyourguide.fr/seminyak-l95096/...,['https://www.booking.com/hotel/id/sanak-retre...
2,https://www.petitfute.com/v43490-caldera-de-ta...,['https://www.getyourguide.fr/tenerife-l350/vi...,[]
3,https://www.petitfute.com/v49846-genes/actuali...,['https://www.getyourguide.fr/genes-l1009/visi...,['https://www.booking.com/hotel/it/ostellin-ge...
4,https://www.petitfute.com/v48986-copenhague/ac...,['https://www.getyourguide.fr/faelledparken-l1...,['https://www.booking.com/hotel/dk/steel-house...


In [34]:
df_liens

,URL,getyourguide_links,booking_links,type
0,https://www.petitfute.com/v62250-kew/actualite...,['https://www.getyourguide.fr/londres-l57/lond...,['https://www.booking.com/hotel/gb/ibis-london...,None
1,https://www.petitfute.com/v48299-bangli/actual...,['https://www.getyourguide.fr/seminyak-l95096/...,['https://www.booking.com/hotel/id/sanak-retre...,None
2,https://www.petitfute.com/v43490-caldera-de-ta...,['https://www.getyourguide.fr/tenerife-l350/vi...,[],None
3,https://www.petitfute.com/v49846-genes/actuali...,['https://www.getyourguide.fr/genes-l1009/visi...,['https://www.booking.com/hotel/it/ostellin-ge...,None
4,https://www.petitfute.com/v48986-copenhague/ac...,['https://www.getyourguide.fr/faelledparken-l1...,['https://www.booking.com/hotel/dk/steel-house...,None
...,...,...,...,...
94,https://www.petitfute.com/v42470-bodrum/actual...,['https://www.getyourguide.fr/bodrum-l846/tour...,['https://www.booking.com/hotel/tr/seckin-kona...,None
95,https://www.petitfute.com/v42420-antalya/actua...,['https://www.getyourguide.fr/antalya-l172/ant...,['https://www.booking.com/hotel/tr/argos.fr.ht...,None
96,https://www.petitfute.com/v41475-chiang-rai/ac...,['https://www.getyourguide.fr/bangkok-l169/ban...,['https://www.booking.com/hotel/th/baan-noppaw...,None
97,https://www.petitfute.com/v48393-seoul/actuali...,['https://www.getyourguide.fr/seoul-l197/palai...,['https://www.booking.com/hotel/kr/grand-hyatt...,None


In [35]:
type(df_liens['booking_links'].iloc[0])


str

In [38]:
import pandas as pd
import re
import ast

# --- 1️⃣ Charger uniquement les colonnes utiles ---
df_liens_tmp = df_liens[['URL', 'booking_links']].copy()

# --- 2️⃣ Convertir les chaînes en vraies listes (si besoin) ---
df_liens_tmp['booking_links'] = df_liens_tmp['booking_links'].apply(
    lambda x: ast.literal_eval(x) if isinstance(x, str) and x.startswith('[') else x
)

# --- 3️⃣ Exploser les listes pour avoir une ligne par lien ---
df_tmp = df_liens_tmp.explode('booking_links').reset_index(drop=True)

# --- 4️⃣ Supprimer les lignes sans lien ---
df_tmp = df_tmp[df_tmp['booking_links'].notna()].copy()

# --- 5️⃣ Extraire le type (ex : hotel, apartment, etc.) ---
def extraire_type_booking(url):
    if isinstance(url, str):
        match = re.search(r'booking\.com/([^/]+)/', url)
        if match:
            return match.group(1)
    return None

df_tmp['type'] = df_tmp['booking_links'].apply(extraire_type_booking)

# --- 6️⃣ Extraire le nom d’établissement (ex : park-hyatt-zurich) ---
def extraire_nom_hotel(url):
    if isinstance(url, str):
        match = re.search(r'booking\.com/hotel/[a-z]{2}/([^/.]+)', url)
        if match:
            return match.group(1)
    return None

df_tmp['nom_hotel'] = df_tmp['booking_links'].apply(extraire_nom_hotel)

# --- 7️⃣ Vérification du résultat ---
df_tmp[['URL', 'booking_links', 'type', 'nom_hotel']].head()


,URL,booking_links,type,nom_hotel
0,https://www.petitfute.com/v62250-kew/actualite...,https://www.booking.com/hotel/gb/ibis-london-b...,hotel,ibis-london-blackfriars
1,https://www.petitfute.com/v62250-kew/actualite...,https://www.booking.com/hotel/gb/ibis-london-b...,hotel,ibis-london-blackfriars
2,https://www.petitfute.com/v62250-kew/actualite...,https://www.booking.com/hotel/gb/therookery.fr...,hotel,therookery
3,https://www.petitfute.com/v62250-kew/actualite...,https://www.booking.com/hotel/gb/sanderson.fr....,hotel,sanderson
4,https://www.petitfute.com/v48299-bangli/actual...,https://www.booking.com/hotel/id/sanak-retreat...,hotel,sanak-retreat-bali


In [ ]:
https://www.booking.com/hotel/pl/novotelwarszawacentrumwarszawa.fr.html?aid=2324560&no_rooms=1&group_adults=2&label=actu-quefaire-varsovie

In [46]:
booking

,Booking date,Booking number,Check-in date,Check-out date,Length of stay,Booking window,Status,Your commission,Commission %,Credit slip,...,Property type,Country,City,UFI,Affiliate ID,Label,Booker country,Booker language,User device,Travel purpose
0,10/8/2025,5362867269,10/27/2025,10/28/2025,1,19,Booked,3.56,40.0,NaN,...,Holiday home,FR,Franquevielle,-1428038,1180643,petitfute-BoxOSV-POI,France,FR,Mobile Browser,LEISURE
1,10/8/2025,5223394350,11/8/2025,11/11/2025,3,31,Booked,20.13,40.0,NaN,...,Hotel,CZ,Prague,-553173,2324560,actu-3jours-prague,Belgium,FR,Desktop,LEISURE
2,10/8/2025,5784114772,12/17/2025,12/19/2025,2,70,Booked,9.81,40.0,NaN,...,Lodge,CR,San Gerardo de Dota,900052668,1180643,conf_email_print,France,FR,Desktop,UNKNOWN
3,10/8/2025,5804108674,12/12/2025,12/14/2025,2,65,Booked,10.74,40.0,NaN,...,Hotel,CR,Dominical,-1107038,1180643,conf_email_print,France,FR,Desktop,UNKNOWN
4,10/8/2025,6162599111,12/10/2025,12/12/2025,2,63,Booked,7.07,40.0,NaN,...,Bed and Breakfast,CR,Quepos,-1108763,1180643,conf_email_print,France,FR,Desktop,UNKNOWN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
84,10/1/2025,5299601560,10/1/2025,10/2/2025,1,0,Booked,10.54,40.0,NaN,...,Hotel,ES,Córdoba,-378765,2324560,actu_quefaire_cordoue,France,FR,App,UNKNOWN
85,10/1/2025,5128273214,10/5/2025,10/6/2025,1,4,Booked,7.75,40.0,NaN,...,Hotel,FR,Saumur,-1469710,1180643,petitfute-bouton_reserver-Autres,Canada,FR,Tablet,LEISURE
86,10/1/2025,5905593911,10/2/2025,10/5/2025,3,1,Booked,10.50,40.0,NaN,...,Hostel,NL,Amsterdam,-2140479,2324560,amsterdam_3_jours,France,FR,App,LEISURE
87,10/1/2025,6755683486,10/1/2025,10/2/2025,1,0,Booked,8.16,40.0,NaN,...,Hotel,FR,Vannes,-1474712,1180643,vizity,France,FR,Desktop,LEISURE


In [45]:
df_tmp[df_tmp['nom_hotel'].str.lower().str.contains('conquistador')]

,URL,booking_links,type,nom_hotel


In [29]:
booking = pd.read_csv(r'C:\Users\Hanaa\Desktop\PETITFUTE\data\exports\bookingcom-Fp693Iv8.csv')

In [30]:
booking

,Booking date,Booking number,Check-in date,Check-out date,Length of stay,Booking window,Status,Your commission,Commission %,Credit slip,...,Property type,Country,City,UFI,Affiliate ID,Label,Booker country,Booker language,User device,Travel purpose
0,10/8/2025,5362867269,10/27/2025,10/28/2025,1,19,Booked,3.56,40.0,NaN,...,Holiday home,FR,Franquevielle,-1428038,1180643,petitfute-BoxOSV-POI,France,FR,Mobile Browser,LEISURE
1,10/8/2025,5223394350,11/8/2025,11/11/2025,3,31,Booked,20.13,40.0,NaN,...,Hotel,CZ,Prague,-553173,2324560,actu-3jours-prague,Belgium,FR,Desktop,LEISURE
2,10/8/2025,5784114772,12/17/2025,12/19/2025,2,70,Booked,9.81,40.0,NaN,...,Lodge,CR,San Gerardo de Dota,900052668,1180643,conf_email_print,France,FR,Desktop,UNKNOWN
3,10/8/2025,5804108674,12/12/2025,12/14/2025,2,65,Booked,10.74,40.0,NaN,...,Hotel,CR,Dominical,-1107038,1180643,conf_email_print,France,FR,Desktop,UNKNOWN
4,10/8/2025,6162599111,12/10/2025,12/12/2025,2,63,Booked,7.07,40.0,NaN,...,Bed and Breakfast,CR,Quepos,-1108763,1180643,conf_email_print,France,FR,Desktop,UNKNOWN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
84,10/1/2025,5299601560,10/1/2025,10/2/2025,1,0,Booked,10.54,40.0,NaN,...,Hotel,ES,Córdoba,-378765,2324560,actu_quefaire_cordoue,France,FR,App,UNKNOWN
85,10/1/2025,5128273214,10/5/2025,10/6/2025,1,4,Booked,7.75,40.0,NaN,...,Hotel,FR,Saumur,-1469710,1180643,petitfute-bouton_reserver-Autres,Canada,FR,Tablet,LEISURE
86,10/1/2025,5905593911,10/2/2025,10/5/2025,3,1,Booked,10.50,40.0,NaN,...,Hostel,NL,Amsterdam,-2140479,2324560,amsterdam_3_jours,France,FR,App,LEISURE
87,10/1/2025,6755683486,10/1/2025,10/2/2025,1,0,Booked,8.16,40.0,NaN,...,Hotel,FR,Vannes,-1474712,1180643,vizity,France,FR,Desktop,LEISURE


In [157]:
df_final[df_final['URL']=='https://www.petitfute.com/v62250-kew/actualite/m17-top-10-insolites-voyage/a11635-que-faire-a-londres-a-noel-les-14-activites-incontournables.html']

,URL,getyourguide_links,booking_links,id_getyourguide_link,revenu_gyg
0,https://www.petitfute.com/v62250-kew/actualite...,[https://www.getyourguide.fr/londres-l57/londr...,['https://www.booking.com/hotel/gb/ibis-london...,"[t41419, t129039, t31392, t310718, t34660, t38...",6.45


In [51]:
import unicodedata

def normaliser_nom(nom):
    if not isinstance(nom, str):
        return ""
    # mettre en minuscules
    nom = nom.lower()
    # remplacer les tirets par des espaces
    nom = nom.replace("-", " ")
    # retirer accents
    nom = ''.join(
        c for c in unicodedata.normalize('NFD', nom)
        if unicodedata.category(c) != 'Mn'
    )
    # retirer espaces multiples
    nom = re.sub(r'\s+', ' ', nom).strip()
    return nom

df_tmp['nom_hotel_norm'] = df_tmp['nom_hotel'].apply(normaliser_nom)
booking['property_name_norm'] = booking['Property name'].apply(normaliser_nom)


In [53]:
df_jointure = df_tmp.merge(
    booking,
    left_on='nom_hotel_norm',
    right_on='property_name_norm',
    how='left',
    indicator=True
)


In [57]:
from rapidfuzz import process, fuzz

# --- 1️⃣ Liste des noms Booking normalisés ---
booking_names = booking['property_name_norm'].dropna().unique().tolist()

# --- 2️⃣ Fonction de recherche du meilleur match ---
def trouver_meilleur_match(nom):
    if not isinstance(nom, str) or not nom:
        return None, 0
    match, score, _ = process.extractOne(
        nom,
        booking_names,
        scorer=fuzz.token_sort_ratio  # bon pour les noms avec mots inversés
    )
    return match, score

# --- 3️⃣ Application sur df_tmp ---
df_tmp[['match_booking', 'score_match']] = df_tmp['nom_hotel_norm'].apply(
    lambda x: pd.Series(trouver_meilleur_match(x))
)

# --- 4️⃣ Vérification ---
df_tmp[['nom_hotel_norm', 'match_booking', 'score_match']].sort_values('score_match', ascending=False).head(20)


,nom_hotel_norm,match_booking,score_match
129,hostal les monges palace,les monges palace boutique,80.000000
87,hostal regio,hostal roma,78.260870
84,hostal regio,hostal roma,78.260870
261,hostal jume,hostal roma,72.727273
26,hotelcasanova,hotel novano,72.000000
279,le petit riad,petit palace vargas,68.750000
198,grandhotelexcelsior,le grand hotel moliere,68.292683
60,modern times amsterdam,travel hotel amsterdam,68.181818
57,modern times amsterdam,travel hotel amsterdam,68.181818
145,villa angela,villa mango b&b,66.666667


In [55]:
print(df_jointure['_merge'].value_counts()
)

_merge
left_only     279
right_only      0
both            0
Name: count, dtype: int64


In [62]:
data_gyg['Booking date'].value_counts()

Booking date
2025-10-07    17
2025-10-08    14
2025-10-05    13
2025-10-03    13
2025-10-01    12
2025-10-04    12
2025-10-06    11
2025-10-02    11
Name: count, dtype: int64

In [154]:
booking_names

['la petite danseuse',
 'hotel julian',
 'los lagos lodge',
 'hotel villas rio mar',
 "seaview studio's, free breakfast & park tour",
 'la suite hotel agadir adults only',
 "la cote des blancs domaine viticole & maison d'hotes",
 'appartements berger kitzbuhel',
 'nh genova centro',
 'the people lyon',
 'les monges palace boutique',
 'gites equestres lou caloun les saintes maries de la mer',
 'le capitaine leparticulier collection',
 "le nid 2 quimper by nid'ouest",
 '"cap riviera" hotel & restaurant saint aygulf',
 'tucantico lodge monteverde',
 'hotel bhoma troyes',
 'tizzy apartment & rooms',
 'villa mango b&b',
 'chez vallye',
 'palacio pinello',
 'le vendome shared appartment',
 'apartamentos dona concha',
 'villas paradise arenal volcano lake view',
 'villas santa maria private bay lake arenal',
 'h2 hotel berlin alexanderplatz',
 'apartamento portbou',
 'urban spa romantique chic',
 'acta antibes',
 'hotel royal',
 'villa cyriel',
 'hotel vice versa',
 'hostel',
 'petit palace v

In [ ]:
df_tmp[df_tmp['nom_hotel_norm'].str.contains('sanderson')]



,URL,booking_links,type,nom_hotel,nom_hotel_norm,match_booking,score_match
3,https://www.petitfute.com/v62250-kew/actualite...,https://www.booking.com/hotel/gb/sanderson.fr....,hotel,sanderson,sanderson,b&b salernow2,45.454545
